# Limpieza y Preprocesamiento de Datos de Manufactura

Este notebook implementa un proceso completo de limpieza y preprocesamiento del dataset original de manufactura. El objetivo principal es eliminar inconsistencias, valores faltantes y preparar los datos para análisis posteriores.

**Archivo de entrada**: `Dataset_Talento.csv`  
**Archivo de salida**: `cleaned_dataset.csv`

> **Proceso**: Incluye limpieza sistemática, análisis temporal y validación de calidad de datos.

# Instalación de Dependencias

Instalamos las librerías necesarias para el procesamiento de datos, análisis estadístico y visualización.

**Librerías requeridas**:
- **pandas**: Manipulación y análisis de datos
- **matplotlib**: Visualización básica de datos  
- **seaborn**: Visualizaciones estadísticas avanzadas
- **scipy**: Análisis estadístico y correlaciones

In [36]:
!pip install pandas matplotlib seaborn scipy



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importación de Librerías Básicas

Importamos las librerías fundamentales para el análisis de datos.

**Librerías utilizadas**:
- **pandas**: Para manipulación de DataFrames y análisis de datos
- **numpy**: Para operaciones numéricas y manejo de arrays

# Carga del Dataset Original

Cargamos el dataset original sin procesar desde el archivo CSV.

**Archivo**: `Dataset_Talento.csv`  
**Contenido**: Datos brutos de manufactura que requieren limpieza y procesamiento

> **Importante**: Este es el punto de partida del proceso de limpieza de datos.

# Inspección Inicial del Dataset

Realizamos una inspección preliminar del dataset para entender su estructura y identificar posibles problemas de calidad de datos.

## Información obtenida:

- **Dimensiones**: Número de filas y columnas
- **Tipos de datos**: Identificación de variables numéricas y categóricas
- **Valores faltantes**: Detección de NaN por columna
- **Memoria utilizada**: Estimación del uso de recursos

> **Objetivo**: Evaluar la calidad inicial de los datos antes del proceso de limpieza.

In [37]:
import pandas as pd
import numpy as np

In [38]:
testing = pd.read_csv("Dataset_Talento.csv")

In [39]:
testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              6000 non-null   object 
 1   turno                  6000 non-null   object 
 2   operador_id            6000 non-null   object 
 3   maquina_id             6000 non-null   object 
 4   producto_id            6000 non-null   object 
 5   temperatura            5820 non-null   float64
 6   vibración              5820 non-null   float64
 7   humedad                5820 non-null   float64
 8   tiempo_ciclo           5820 non-null   float64
 9   fallo_detectado        6000 non-null   object 
 10  tipo_fallo             599 non-null    object 
 11  cantidad_producida     6000 non-null   int64  
 12  unidades_defectuosas   6000 non-null   int64  
 13  eficiencia_porcentual  5820 non-null   float64
 14  consumo_energia        5820 non-null   float64
 15  para

# Proceso de Limpieza de Datos

Esta sección implementa el algoritmo principal de limpieza de datos, eliminando sistemáticamente las filas que contienen valores faltantes en variables críticas para el análisis.

## Estrategia de limpieza:

1. **Definición de variables críticas**: Identificación de columnas esenciales
2. **Evaluación de completitud**: Análisis de valores faltantes por variable
3. **Eliminación selectiva**: Remoción de registros con datos incompletos
4. **Validación de resultados**: Verificación de la calidad post-limpieza

> **Enfoque**: Eliminación de filas con valores faltantes en lugar de imputación para mantener la integridad de los datos.

In [40]:
testing = pd.read_csv("Dataset_Talento.csv")

print("=== DATA CLEANING: REMOVING ROWS WITH NaN IN SPECIFIC COLUMNS ===\n")

# Define the columns that cannot have NaN values
critical_columns = ['temperatura', 'vibración', 'humedad', 'tiempo_ciclo', 'eficiencia_porcentual', 'consumo_energia']

print(f"Critical columns (cannot have NaN): {critical_columns}")
print("=" * 60)

# Get initial dataset info
initial_rows = len(testing)
print(f"Initial dataset: {initial_rows} rows")

# Check which critical columns exist in the dataset
existing_critical_cols = [col for col in critical_columns if col in testing.columns]
missing_critical_cols = [col for col in critical_columns if col not in testing.columns]

print(f"\nFound critical columns: {existing_critical_cols}")
if missing_critical_cols:
    print(f"Missing critical columns: {missing_critical_cols}")

# Show NaN count in critical columns before cleaning
print(f"\nNaN count in critical columns BEFORE cleaning:")
for col in existing_critical_cols:
    nan_count = testing[col].isna().sum()
    print(f"  {col}: {nan_count} NaN values")

# Remove rows with NaN in any of the existing critical columns
if existing_critical_cols:
    testing_clean = testing.dropna(subset=existing_critical_cols).reset_index(drop=True)
else:
    print("No critical columns found in dataset - no cleaning performed")
    testing_clean = testing.copy()

# Calculate cleaning results
final_rows = len(testing_clean)
removed_rows = initial_rows - final_rows

print(f"\nCLEANING RESULTS:")
print("=" * 40)
print(f"Rows before cleaning: {initial_rows}")
print(f"Rows after cleaning:  {final_rows}")
print(f"Rows removed:         {removed_rows}")
print(f"Data retention:       {(final_rows/initial_rows*100):.1f}%")

# Show overall dataset info after cleaning
print("\n" + "=" * 80)
print("DATASET INFO AFTER CLEANING:")
print("=" * 80)
for i, col in enumerate(testing_clean.columns, 1):
    dtype = testing_clean[col].dtype
    non_null = testing_clean[col].count()
    total = len(testing_clean)
    null_count = total - non_null
    null_pct = (null_count/total*100) if total > 0 else 0

    # Mark critical columns
    marker = " ★" if col in existing_critical_cols else ""

    print(f"{i:2d}. {col:25s} | {str(dtype):10s} | {non_null:4d}/{total} non-null | {null_count:3d} NaN ({null_pct:5.1f}%){marker}")


=== DATA CLEANING: REMOVING ROWS WITH NaN IN SPECIFIC COLUMNS ===

Critical columns (cannot have NaN): ['temperatura', 'vibración', 'humedad', 'tiempo_ciclo', 'eficiencia_porcentual', 'consumo_energia']
Initial dataset: 6000 rows

Found critical columns: ['temperatura', 'vibración', 'humedad', 'tiempo_ciclo', 'eficiencia_porcentual', 'consumo_energia']

NaN count in critical columns BEFORE cleaning:
  temperatura: 180 NaN values
  vibración: 180 NaN values
  humedad: 180 NaN values
  tiempo_ciclo: 180 NaN values
  eficiencia_porcentual: 180 NaN values
  consumo_energia: 180 NaN values

CLEANING RESULTS:
Rows before cleaning: 6000
Rows after cleaning:  4991
Rows removed:         1009
Data retention:       83.2%

DATASET INFO AFTER CLEANING:
 1. timestamp                 | object     | 4991/4991 non-null |   0 NaN (  0.0%)
 2. turno                     | object     | 4991/4991 non-null |   0 NaN (  0.0%)
 3. operador_id               | object     | 4991/4991 non-null |   0 NaN (  0.0%)
 

In [41]:
testing_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4991 entries, 0 to 4990
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              4991 non-null   object 
 1   turno                  4991 non-null   object 
 2   operador_id            4991 non-null   object 
 3   maquina_id             4991 non-null   object 
 4   producto_id            4991 non-null   object 
 5   temperatura            4991 non-null   float64
 6   vibración              4991 non-null   float64
 7   humedad                4991 non-null   float64
 8   tiempo_ciclo           4991 non-null   float64
 9   fallo_detectado        4991 non-null   object 
 10  tipo_fallo             495 non-null    object 
 11  cantidad_producida     4991 non-null   int64  
 12  unidades_defectuosas   4991 non-null   int64  
 13  eficiencia_porcentual  4991 non-null   float64
 14  consumo_energia        4991 non-null   float64
 15  para

# Análisis Temporal y de Correlaciones

En esta sección realizamos un análisis avanzado de los datos limpios, enfocándonos en patrones temporales y correlaciones entre variables de manufactura.

## Análisis implementados:

1. **Extracción de características temporales**: Hora, día de la semana, mes
2. **Análisis de correlaciones temporales**: Relaciones entre tiempo y variables operacionales  
3. **Visualizaciones temporales**: Gráficos de rendimiento por períodos
4. **Identificación de patrones**: Detección de tendencias y ciclos

> **Objetivo**: Descubrir patrones temporales que puedan influir en la eficiencia y calidad de la producción.

# Validación de Datos: Eficiencia Porcentual

Identificamos y analizamos valores anómalos en la variable de eficiencia porcentual.

## Validación realizada:

- **Detección de anomalías**: Identificación de valores de eficiencia > 100%
- **Análisis de distribución**: Evaluación de la frecuencia de valores anómalos
- **Evaluación de impacto**: Determinación del efecto en el análisis

> **Criterio**: La eficiencia porcentual no debería exceder el 100% en condiciones normales de operación.

In [42]:
testing = testing_clean.copy

In [43]:
testing_clean[testing_clean['eficiencia_porcentual'] > 100]['eficiencia_porcentual']

298     101.573110
693     100.179076
1957    102.024829
2053    101.016465
4300    100.520575
Name: eficiencia_porcentual, dtype: float64

In [44]:
testing_cleaned = testing_clean.query('eficiencia_porcentual <= 100')
testing_cleaned[testing_cleaned['eficiencia_porcentual'] > 100]['eficiencia_porcentual']

Series([], Name: eficiencia_porcentual, dtype: float64)

In [45]:
testing_cleaned.to_csv("cleaned_dataset.csv", index=False)
testing_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4986 entries, 0 to 4990
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              4986 non-null   object 
 1   turno                  4986 non-null   object 
 2   operador_id            4986 non-null   object 
 3   maquina_id             4986 non-null   object 
 4   producto_id            4986 non-null   object 
 5   temperatura            4986 non-null   float64
 6   vibración              4986 non-null   float64
 7   humedad                4986 non-null   float64
 8   tiempo_ciclo           4986 non-null   float64
 9   fallo_detectado        4986 non-null   object 
 10  tipo_fallo             495 non-null    object 
 11  cantidad_producida     4986 non-null   int64  
 12  unidades_defectuosas   4986 non-null   int64  
 13  eficiencia_porcentual  4986 non-null   float64
 14  consumo_energia        4986 non-null   float64
 15  paradas_p